## Análisis de Componentes Principales
* Estandarizar los datos para cada una de las columnas.
* Realizar la reducción del dataset.
* Transformar el dataset original para así obtener datos en el subespacio dimensional.

In [1]:
import pandas as pd # Para manejar los datos

In [2]:
# El df está configurado con un encoding diferente, con este código lo averiguamos
# abrimos el archivo con open(ruta archivo, "r") y lo asignamos a una variable
df = open("./Ventas_Videojuegos.csv","r") 
# leemos el objeto df --> encoding='cp1252'
df
# cerramos el open
df.close()

In [3]:
# Cargamos el csv en una variable y verificamos
df = pd.read_csv("./Ventas_Videojuegos.csv", encoding = "cp1252", sep = ";")
df.head()

,Nombre,Plataforma,Año,Genero,Editorial,Ventas NA,Ventas EU,Ventas JP,Ventas Otros,Ventas Global
0,Wii Sports,Wii,2006,Sports,Nintendo,"41,49","29,02","3,77","8,46","82,74"
1,Super Mario Bros.,NES,1985,Platform,Nintendo,"29,08","3,58","6,81","0,77","40,24"
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,"15,85","12,88","3,79","3,31","35,82"
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,"15,75","11,01","3,28","2,96",33
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,"11,27","8,89","10,22",1,"31,37"


In [ ]:
# Para realizar el Análisis de Componentes Principales
from sklearn.decomposition import PCA 
# Para estandarizar los datos (escalarlos a media 0 y desviación estándar 1)
from sklearn.preprocessing import StandardScaler 
# Para operaciones numéricas y matemáticas avanzadas
import numpy as np 

In [5]:
# Seleccionamos las columnas con valores numéricos para el ACP
cols = ['Ventas NA', 'Ventas EU', 'Ventas JP', 'Ventas Otros', 'Ventas Global']
X = df[ cols ]
# Verificar si hay valores nulos y manejarlos
X = X.fillna(0)  # En este caso lo reemplazamos con 0, pero si queremos se puede sacar el promedio o otro método
#Convertimos todos los datos a tipo float
for col in cols:
     X[col] = X[col].astype(str).str.replace(',', '.', regex = False).astype(float)

In [ ]:
# Crea una instancia de StandardScaler. 
# Esto evita que las variables con mayores rangos de valores dominen el cálculo de los componentes.
scaler = StandardScaler()
xEstandarizado = scaler.fit_transform( X ) # Aplicamos la standarizacion, a las columnas numéricas para el PCA

In [7]:
# Elegimos el número de componentes.
pca = PCA( n_components = 5 )

In [8]:
# Ajustamos el modelo PCA a los datos estandarizados y los transformamos
componentesPrincipales = pca.fit_transform( xEstandarizado )

### Explicamos los 2 valores en un gráfico
* La varianza explicada y la varianza explicada acumulada

In [9]:
import plotly.graph_objs as go
#Utilizamos plotly para gráficos interactivos

In [10]:
plot1 = go.Bar( x = ["CP %s"%i for i in range(1, 6)], y = pca.explained_variance_ratio_, showlegend = False ) #CP = componente principal
plot2 = go.Scatter( x = ["CP %s"%i for i in range(1, 6)], y = np.cumsum( pca.explained_variance_ratio_ ), showlegend = True, name = "% de varianza explicada acumulada" )
data = go.Data([ plot1, plot2 ])
layout = go.Layout( xaxis = go.XAxis( title = "Componentes principales"),
                    yaxis = go.YAxis( title = "% de varianza explicada" ),
                    title = "% de variabilidad explicada por cada componente principal" 
                    )
fig = go.Figure( data, layout )
fig.show()

c:\Users\Cliente\anaconda3\Lib\site-packages\plotly\graph_objs\_deprecations.py:31: DeprecationWarning: plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.

  warnings.warn(
c:\Users\Cliente\anaconda3\Lib\site-packages\plotly\graph_objs\_deprecations.py:544: DeprecationWarning: plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis

  warnings.warn(
c:\Users\Cliente\anaconda3\Lib\site-packages\plotly\graph_objs\_deprecations.py:572: DeprecationWarning: plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis

  warnings.warn(


In [11]:
# Verificamos la varianza explicada acumulada por los componentes seleccionados
varExpCum = np.cumsum( pca.explained_variance_ratio_ )
print("Varianza explicada acumulada en %:", varExpCum)

Varianza explicada acumulada en %: [0.73572418 0.88671091 0.95816552 0.99999845 1.        ]


* Observamos en el gráfico que utilizando 3 de los 5 componentes obtenemos el 95 % de los datos.
* Lo que considero suficiente para poder recortar gran parte del dataset sin alterar su funcionamiento.

### Reducción de componentes
* Estandarizamos nuevamente para recortar nuestros datos.

In [12]:
# Elegimos el número de componentes.
pca = PCA( n_components = 3 )
# Ajustamos el modelo PCA a los datos estandarizados y los transformamos
componentesPrincipales = pca.fit_transform( xEstandarizado )

In [13]:
# Creamos un DataFrame con los componentes principales
# Los nombres de las columnas serán PC1, PC2, PC3
dfReducido = pd.DataFrame( data = componentesPrincipales,
                           columns=['PC1', 'PC2', 'PC3'] )

print("DataFrame con los Componentes Principales:\n", dfReducido.head())
# Verificamos la varianza explicada y la varianza acumulada explicada por los componentes seleccionados
varExp = pca.explained_variance_ratio_
varExpCum = np.cumsum( pca.explained_variance_ratio_ )
print("\nVarianza explicada por cada componente en %:", pca.explained_variance_ratio_ )
print("Varianza explicada acumulada en %:", np.cumsum( pca.explained_variance_ratio_ ))


DataFrame con los Componentes Principales:
           PC1        PC2        PC3
0  100.445216  20.631828  -6.802817
1   41.484198 -14.451379 -16.608134
2   43.638878   1.974957  -0.704624
3   39.593035   2.026401  -1.936774
4   37.306433 -23.156630   0.941124

Varianza explicada por cada componente en %: [0.73572418 0.15098673 0.07145462]
Varianza explicada acumulada en %: [0.73572418 0.88671091 0.95816552]


In [ ]:
# Guardamos el DataFrame a un archivo CSV
# index=False es importante para no guardar el índice del DataFrame como una columna en el CSV
dfReducido.to_csv( "dfReducido.csv", index = False )

#### El resto del código es solo por si queremos ver los valores que contiene PC1 PC2 y PC3

In [14]:
# Obtenemos los loadings (cargas)
pca.components_ 
# Contiene los vectores propios (eigenvectors) que representan las cargas de cada variable original en cada componente.

array([[ 0.47284015,  0.47694446,  0.32155877,  0.42088079,  0.51807867],
       [ 0.0718796 ,  0.17448217, -0.88199007,  0.4308726 , -0.02883775],
       [-0.59507165, -0.05121852,  0.29972299,  0.72137361, -0.18180384]])

In [ ]:
# Generamos un DataFrame por si queremos examinar los datos dentro de PC1 PC2 y PC3
dfLoadings = pd.DataFrame( 
                            pca.components_,
                            columns = X.columns,
                            index=['PC1', 'PC2', 'PC3']
                        )
print("--- Cargas (Loadings) de las Variables Originales en cada Componente Principal ---")
print( dfLoadings )

print("\n--- Varianza Explicada por cada Componente ---")
print("Varianza explicada por cada componente:", varExp )
print("Varianza explicada acumulada:", varExpCum )
# El PC1 representa el volumen general de ventas, ya que las ventas en todas las regiones tienen una alta carga positiva en él.

--- Cargas (Loadings) de las Variables Originales en cada Componente Principal ---
     Ventas NA  Ventas EU  Ventas JP  Ventas Otros  Ventas Global
PC1   0.472840   0.476944   0.321559      0.420881       0.518079
PC2   0.071880   0.174482  -0.881990      0.430873      -0.028838
PC3  -0.595072  -0.051219   0.299723      0.721374      -0.181804

--- Varianza Explicada por cada Componente ---
Varianza explicada por cada componente: [0.73572418 0.15098673 0.07145462]
Varianza explicada acumulada: [0.73572418 0.88671091 0.95816552]
